In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import selenium
print(selenium.__version__)

4.8.2


# 4Chan

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options as EdgeOptions
from pathlib import Path

downloads_dir = Path("downloads")
downloads_dir.mkdir(exist_ok=True)
# from msedge.selenium_tools import EdgeOptions
options = EdgeOptions()
options.add_experimental_option("prefs", {
    "download.default_directory": str(downloads_dir),
    "download.directory_upgrade": True,
})
# options.add_argument("--headless")
# options.add_argument("--disable-gpu") 
browser = webdriver.Edge(options=options)
# browser.set_window_position(1920,0)
browser.get("https://boards.4channel.org/g/thread/92468569#p92471725")
# href = browser.find_element(By.CSS_SELECTOR, "a[title='Mdk']").get_attribute("href")
# href = href.split("&url=")[1]
# browser.get(href)
# browser.close()
# del browser
# print(f"href: {href}")

In [17]:
result = browser.execute_script(r"return {a:4, b:6, myName:'is daddy'};")
print(result) # holy shit it converts the objects

{'a': 4, 'b': 6, 'myName': 'is daddy'}


In [ ]:
import requests
url = "http://127.0.0.1:5000/"
#     prompt = f"""The following is a conversation between a human and an AI.
# Human: {human_input}
# AI:"""
prompt=f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{human_input}

### Context:
{human_input}

### Response:"""
data = {"prompts": [prompt]}
response = requests.post(url, json=data).json()[0]
response = response.replace(prompt, "")
if "Human:" in response:
    response = response.split("Human:")[0]
response = response.strip()
return response
# await get_response("Hi there!")

# Picker

In [50]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Edge()
driver.get("https://twitter.com")

In [60]:
async def getSelector():
    print("Prompting user to click")
    result = driver.execute_async_script(r"""
var done = arguments[0];
const panel = document.createElement('div');
panel.style.position = 'fixed';
panel.style.top = '0';
panel.style.left = '0';
panel.style.width = '100%';
panel.style.height = '100%';
panel.style.zIndex = '9999';
panel.style.background = 'rgba(0,0.5,0,0.5)';
panel.innerText = "CLICK THE ELEMENT.";
panel.style.color = 'white';
panel.style.fontSize = '50px';
panel.style.fontFamily = 'Impact, Charcoal, sans-serif';

panel.style.display = 'flex';
panel.style.justifyContent = 'center';
panel.style.alignItems = 'center';
panel.style.cursor = 'pointer';

function preprocess(list, keyGetter, priority, disallowPred) {
  return Array.from(list).filter(x => !disallowPred(keyGetter(x))).sort((a, b) => {
    a = keyGetter(a);
    b = keyGetter(b);
    const aInPriority = priority.includes(a);
    const bInPriority = priority.includes(b);

    if (aInPriority && bInPriority) {
      return 0;
    } else if (aInPriority) {
      return -1;
    } else if (bInPriority) {
      return 1;
    } else {
      return a.localeCompare(b);
    }
  });
}

function getXPath(element) {
  function getUniqueXPath(el) {
    while (el.childNodes.length === 1 && el.firstChild.nodeType === 1) el = el.firstChild;
    let tagName = el.tagName.toLowerCase();

    function isUnique(xpath) {
      return el.parentElement.querySelectorAll(xpath).length === 1;
    }

    let xpath = `./${tagName}`;
    if (el.id) {
      xpath += `[@id='${el.id}']`;
      if (isUnique(xpath)) return xpath;
    }

    for (const a of preprocess(element.attributes, x => x.name, ["autocomplete", "href"], x=>false))) {
      if (a.name === 'id' || a.name === 'class' || a.name === 'style') continue;
      xpath += `[@${a.name}='${a.value}']`;
      if (isUnique(xpath)) return xpath;
    }

    if (el.classList.length > 0) {
      for (const className of preprocess(element.classList, x => x, ["content"], x=>x.startsWith("css-") || x.startsWith("r-"))) {
        xpath += `[contains(concat(' ', normalize-space(@class), ' '), ' ${className} ')]`;
        if (isUnique(xpath)) return xpath;
      }
    }

    if (el.textContent) {
      xpath += `[contains(., '${el.textContent.trim()}')]`;
      if (isUnique(xpath)) return xpath;
    }

    const index = Array.from(el.parentNode.children).filter((sibling) => sibling.tagName.toLowerCase() === tagName).indexOf(el);
    xpath = `./${tagName}[${index + 1}]`;

    return xpath;
  }

  const xpaths = [];
  let current = element;

  while (current.parentElement) {
    xpaths.unshift(getUniqueXPath(current));
    current = current.parentElement;
  }

  return '/' + xpaths.join('/');
}

//function getUniqueSelector(element) {
//  while (element.childNodes.length === 1 && element.firstChild.nodeType === 1) element = element.firstChild;
//  let selector = element.tagName.toLowerCase();
//  function isUnique() {
//      return [...element.parentElement.children].filter((sibling) => sibling.matches(selector)).length === 1;
//  }
//  
//  if (element.id) {
//    selector += `#${element.id}`;
//    if (isUnique()) return selector;
//  }
//  for (const a of preprocess(element.attributes, x => x.name, ["autocomplete", "href"], x=>false)) {   
//      if (a.name === 'id' || a.name === 'class' || a.name === "style") continue;
//      selector += `[${a.name}="${a.value}"]`;
//      if (isUnique()) return selector;
//  }
//
//  if (element.classList.length > 0) {
//    for (const className of preprocess(element.classList, x => x, ["content"], x=>x.startsWith("css-") || x.startsWith("r-"))) {
//      selector += `.${className}`;
//      if (isUnique()) return selector;
//    }
//  }
//
//  const siblingSameType = [...element.parentElement.children].filter(
//    (sibling) => sibling.tagName.toLowerCase() === selector
//  );
//  if (siblingSameType.length === 1) return selector;
//  const index = siblingSameType.indexOf(element);
//  selector += `:nth-of-type(${index + 1})`;
//
//  return selector;
//}

function getSelector(el) {
  const elements = [];
  let current = el;

  while (current.parentElement) {
    elements.unshift(current);
    current = current.parentElement;
  }

  return elements.map(getXPath).join(' > ');
}


panel.addEventListener('click', e => {
    // get x,y
    let x = e.clientX;
    let y = e.clientY;
    console.log("clicked", x, y);
    document.body.removeChild(panel);
    const element = document.elementFromPoint(x, y);
    try {
        console.log(element, getSelector(element));
        done(getSelector(element));
    } catch (e) {
        console.error(e);
        done(null);
    }
});
document.body.appendChild(panel);

    """
    )
    print(f"Found element with selector {result}")

    import requests
    url = "http://127.0.0.1:5000/"
    PROMPT_DICT = {
        "prompt_input": (
            "Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
        ),
        "prompt_no_input": (
            "Below is an instruction that describes a task. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Response:"
        ),
    }
    def fmt(prompt: str, input: str = "") -> str:
        if input.strip() == "":
            return PROMPT_DICT['prompt_no_input'].format_map({'instruction': prompt})
        else:
            return PROMPT_DICT['prompt_input'].format_map({'instruction': prompt, 'input': input})
        
    prompts=[
    fmt(
        """You are a calibration engine for a web interface.
    The user has just indicated the log in button on a webpage.
    Clean up the CSS selector for the log in button.
    Remove any unnecessary parts, and make sure it is as short as possible.
    You do not need to start at the root element.
    Try and make it as short as possible.

    ### Example Input:
    body > div#react-root > div > div:n-th-of-type(2) > div > a[href="/home"] > div

    ### Example Output:
    a[href="/home"]
    """,

        result),
    ]
    data = {"prompts": prompts}
    print("Refining selector with remote model")
    response = requests.post(url, json=data).json()
    selector = response[0].split("### Response:")[1].strip()

    assert driver.find_element(By.CSS_SELECTOR, selector) is not None
    return selector


In [61]:
def click(selector):
    driver.execute_script(f"document.querySelector('{selector}').click()")

In [62]:
def focus(selector):
    driver.execute_script(f"document.querySelector('{selector}').focus()")

In [64]:
logInSelector = await getSelector()
print(logInSelector)
click(logInSelector)

Prompting user to click
Found element with selector body > div > div > div > div > div > div > div > div > div > div > div > div > div > span > span > span > span > span
Refining selector with remote model


InvalidSelectorException: Message: invalid selector: An invalid or illegal selector was specified
  (Session info: MicrosoftEdge=111.0.1661.54)
Stacktrace:
Backtrace:
	Microsoft::Applications::Events::EventProperties::SetProperty [0x00007FF7238D3C22+14194]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF72386AEC2+817458]
	(No symbol) [0x00007FF7234FDB50]
	(No symbol) [0x00007FF72350195D]
	(No symbol) [0x00007FF7235015D9]
	(No symbol) [0x00007FF723501A30]
	(No symbol) [0x00007FF72354011B]
	(No symbol) [0x00007FF72354070C]
	(No symbol) [0x00007FF72357C277]
	(No symbol) [0x00007FF7235606CF]
	(No symbol) [0x00007FF723533121]
	(No symbol) [0x00007FF7235796FE]
	(No symbol) [0x00007FF723560343]
	(No symbol) [0x00007FF723531FFC]
	(No symbol) [0x00007FF723530F7E]
	(No symbol) [0x00007FF7235327EE]
	Microsoft::Applications::Events::EventProperties::SetProperty [0x00007FF72374EAA0+180080]
	(No symbol) [0x00007FF723600B17]
	Microsoft::Applications::Events::EventProperty::Ev

In [16]:
usernameSelector = await getSelector()
print(usernameSelector)
focus(usernameSelector)

Prompting user to click
Refining selector with remote model
input[autocomplete="username"]


In [18]:
import mss
import pyautogui
from IPython.display import display
from PIL import Image
def getMouseMonitorScreenshot():
    # Get the current mouse position
    mouse_x, mouse_y = pyautogui.position()

    # Determine the monitor the mouse is on
    with mss.mss() as sct:
        target_monitor = None
        for monitor in sct.monitors[1:]:
            if monitor["left"] <= mouse_x < monitor["left"] + monitor["width"] and monitor["top"] <= mouse_y < monitor["top"] + monitor["height"]:
                target_monitor = monitor
                break

        if target_monitor:
            image = sct.grab(target_monitor)
            return Image.frombytes("RGB", image.size, image.bgra, "raw", "BGRX")
        else:
            return None

In [5]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
def recognize_text(image):
    return pytesseract.image_to_string(image, lang="eng")


'J File Edit Selection view Go ‘© browser.ipynb - toolformer (Workspace) - Visual Studio Code oOegonr - a x\n| EXPLORER ++ browseripynb U @ @ README.md U B® install-dependencies.ps1 M @#aunoao-\n\\ VS CODE PETS © backend > = browseripynb > m#Picker > CHimport pytesseract\n“ ” ff WY, 4+ code + Markdown | [> Run All = Clear all Outputs ‘D Restart | --- G1 venv (Python 3.10.9) =\n\n(7) Xavier\n(ack) on T...\n\n\\ OPEN EDITORS |1 unsaved\n\nShh B- @\n\n(© browseripynb backend U De\n@ READMEMmd b: u\nBF install-dependencies ps... M\n\\ TOOLFORMER (WORKSPACE)\n> B& downloads\n@ lama-7b-hf\nfe models\n@ mio-small 4 © 00s Python\n& old\npythia-2.88-deduped\npythia-6.98-deduped\n\nimport pytesseract\npytesseract.pytesseract.tesseract_cmd = r"C: \\Program Files\\Tesseract-(\n\na\n2\n3 def recognize text (image):\n4 | return pytesseract.image_to_string(image, lang-"eng")\n5\n6\n\n4%\n2\nbie\n>\n1m)\n\nqo Pacognize text (getMouseHonitorScreenshot())\n7]\n\nA\n\ngitignore\n\nbrowseripynb u\n\nconve

In [21]:
sleep(1)
focus(usernameSelector)

In [23]:
import pyautogui
from time import sleep
sleep(1)
pyautogui.hotkey("ctrl","shift","space")
sleep(0.5)
text = recognize_text(getMouseMonitorScreenshot())
if "Search 1Password" in text:
    pyautogui.typewrite("twitter")
    pyautogui.hotkey("ctrl","c")
    focus(usernameSelector)
    sleep(0.5)
    pyautogui.hotkey("ctrl","v")
    

In [28]:
print(loginNextSelector)
driver.execute_script(f"return document.querySelector(`{loginNextSelector}`) || 'not found'")

a[href="/home"]


'not found'

In [33]:
loginNextSelector = await getSelector()
print(loginNextSelector)
click(loginNextSelector)

Prompting user to click
Found element with selector body[data-innertext=""] > div[aria-hidden="false"] > div[aria-hidden="false"] > div[aria-hidden="false"] > div[aria-hidden="false"] > div[data-innertext=""] > div[data-innertext=""] > div[data-innertext=""] > div[data-innertext=""] > div[data-innertext=""] > div[data-innertext=""] > div[data-innertext=""] > span[data-innertext=""] > span[data-innertext=""]
Refining selector with remote model
body[data-innertext=""] > div[aria-hidden="false"] > div[aria-hidden="false"] > div[aria-hidden="false"] > div[aria-hidden="false"] > div[data-innertext=""] > div[data-innertext=""] > div[data-innertext=""] > div[data-innertext=""] > div[data-innertext=""] > span[data-innertext=""] > span[data-innertext=""]


JavascriptException: Message: javascript error: Cannot read properties of null (reading 'click')
  (Session info: MicrosoftEdge=111.0.1661.54)
Stacktrace:
Backtrace:
	Microsoft::Applications::Events::EventProperties::SetProperty [0x00007FF7238D3C22+14194]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF72386AEC2+817458]
	(No symbol) [0x00007FF7234FDB50]
	(No symbol) [0x00007FF72350195D]
	(No symbol) [0x00007FF7235015D9]
	(No symbol) [0x00007FF723502640]
	(No symbol) [0x00007FF72357A692]
	(No symbol) [0x00007FF72356068A]
	(No symbol) [0x00007FF723533121]
	(No symbol) [0x00007FF7235796FE]
	(No symbol) [0x00007FF723560343]
	(No symbol) [0x00007FF723531FFC]
	(No symbol) [0x00007FF723530F7E]
	(No symbol) [0x00007FF7235327EE]
	Microsoft::Applications::Events::EventProperties::SetProperty [0x00007FF72374EAA0+180080]
	(No symbol) [0x00007FF723600B17]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF7237AF772+49634]
	Microsoft::Applications::Even

# Tweet gatherer

In [ ]:
program = """
Go to twitter.com.
Check if the user is logged in. If there is """

In [37]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

driver = webdriver.Edge()
driver.get("https://twitter.com")


# Forge MDK

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options as EdgeOptions
from pathlib import Path

downloads_dir = Path("downloads")
downloads_dir.mkdir(exist_ok=True)
# from msedge.selenium_tools import EdgeOptions
options = EdgeOptions()
options.add_experimental_option("prefs", {
    "download.default_directory": str(downloads_dir),
    "download.directory_upgrade": True,
})
# options.add_argument("--headless")
# options.add_argument("--disable-gpu") 
browser = webdriver.Edge(options=options)
browser.set_window_position(1920,0)
browser.get("https://files.minecraftforge.net/")
href = browser.find_element(By.CSS_SELECTOR, "a[title='Mdk']").get_attribute("href")
href = href.split("&url=")[1]
browser.get(href)
# browser.close()
# del browser
print(f"href: {href}")

href: https://maven.minecraftforge.net/net/minecraftforge/forge/1.19.3-44.1.23/forge-1.19.3-44.1.23-mdk.zip


In [ ]:
%cd ../../src
from tools.forge_mdk import ForgeMDKTool
from tools.helpers import handle_tool_invocation
# tool = ForgeMDKTool()
# tool.invoke("latest")
await handle_tool_invocation("[EDGE(open https://files.minecraftforge.net/) ->")

d:\Repos\Other\toolformer\backend\src
hehe


Traceback (most recent call last):
  File "d:\Repos\Other\toolformer\backend\venv\lib\site-packages\selenium\webdriver\common\service.py", line 72, in start
    self.process = subprocess.Popen(cmd, env=self.env,
  File "C:\Users\TeamD\.conda\envs\3.10\lib\subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\TeamD\.conda\envs\3.10\lib\subprocess.py", line 1440, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
FileNotFoundError: [WinError 2] The system cannot find the file specified

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "d:\Repos\Other\toolformer\backend\src\tools\helpers.py", line 35, in handle_tool_invocation
    result = tool.method(params)
  File "d:\Repos\Other\toolformer\backend\src\tools\edge.py", line 31, in invoke
    driver = Edge(options=options)
  File "d:\Repos\Other\toolformer\backend\venv\lib\site-packa

(True,
 "ERROR: Message: 'msedgedriver' executable needs to be in PATH. Please download from http://go.microsoft.com/fwlink/?LinkId=619687\n")